# Распознавание рукописных цифр

## Часть 1, необработанное изображение и его сдвиги/повороты

Рассмотрим работу RandomForest из 250 деревьев на необработанных изображениях, здесь фичами выступают уровни насыщенности пикселей, всего в картинке 28х28, соответственно, 784 фичи. Обучение проводится на 60000 картинках, валидация на 10000.

In [1]:
import idx2numpy
import numpy
from import_labs import import_labs
from time import time
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    from sklearn.ensemble import RandomForestClassifier as RFC

__author__ = 'vks'

import_labs(["task4p1/", "task3/"])
from random_forest import RandomForest

train_images = numpy.reshape(idx2numpy.convert_from_file("train-images.idx3-ubyte"), (60000, 28 * 28))
train_labels = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
test_images = numpy.reshape(idx2numpy.convert_from_file("t10k-images.idx3-ubyte"), (10000, 28 * 28))
test_labels = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")

forest = RFC(n_estimators=250)

start = time()
forest.fit(train_images, train_labels)

checkpoint = time()
print("*** Learning took %s s." % (checkpoint - start))

errors = 0
for i in range(len(test_labels)):
    if forest.predict(test_images[i]) != test_labels[i]:
        errors += 1

finish = time()
print("*** Validating took %s s. Overall time: %s s. Errors: %s (%s%%)\n\n" %
      (finish - checkpoint, finish - start, errors, errors / 100))

*** Learning took 94.29261350631714 s.
*** Validating took 114.98983502388 s. Overall time: 209.28244853019714 s. Errors: 276 (2.76%)




Посмотрим теперь, как будут классифицированы повёрнутые, отзеркаленные и сдвинутые по обеим осям изображения

In [2]:
from main import *

forest = RFC(n_estimators=250)
forest.fit(numpy.reshape(train_images, (60000, 28 * 28)), train_labels)

validate(forest, numpy.reshape(img_shift_x0y1, (10000, 28 * 28)), "shifted by 1 px up image")
validate(forest, numpy.reshape(img_shift_x0y5, (10000, 28 * 28)), "shifted by 5 px up image")
validate(forest, numpy.reshape(img_shift_x0y10, (10000, 28 * 28)), "shifted by 10 px up image")
validate(forest, numpy.reshape(img_shift_x1y0, (10000, 28 * 28)), "shifted by 1 px left image")
validate(forest, numpy.reshape(img_shift_x5y0, (10000, 28 * 28)), "shifted by 5 px left image")
validate(forest, numpy.reshape(img_shift_x10y0, (10000, 28 * 28)), "shifted by 10 px left image")
validate(forest, numpy.reshape(img_rotate, (10000, 28 * 28)), "rotated image")
validate(forest, numpy.reshape(img_flip_x, (10000, 28 * 28)), "flipped by Ox image")
validate(forest, numpy.reshape(img_flip_y, (10000, 28 * 28)), "flipped by Oy image")

*** Validating shifted by 1 px up image took 124.53428816795349 s. Errors: 720 (7.2%)

*** Validating shifted by 5 px up image took 121.4572274684906 s. Errors: 8922 (89.22%)

*** Validating shifted by 10 px up image took 122.06629514694214 s. Errors: 8629 (86.29%)

*** Validating shifted by 1 px left image took 121.90059328079224 s. Errors: 461 (4.61%)

*** Validating shifted by 5 px left image took 122.50000262260437 s. Errors: 8404 (84.04%)

*** Validating shifted by 10 px left image took 119.19437170028687 s. Errors: 8700 (87.0%)

*** Validating rotated image took 120.17110466957092 s. Errors: 8302 (83.02%)

*** Validating flipped by Ox image took 123.39570021629333 s. Errors: 6710 (67.1%)

*** Validating flipped by Oy image took 126.48140478134155 s. Errors: 5686 (56.86%)



Как видно, если сдвиг изображения на один пиксель ещё не сильно ухудшает результаты, то остальные преобразования изображения делают невозможной его адекватную классификацию.

## Часть 2, изображение, преобразованное в гистограмму направленных градиентов

Теперь лес будет обучаться на преобразованных изображениях, а валидация будет происходить также на преобразованных неизменённых, сдвинутых, повёрнутых и отзеркаленных изображениях.

In [5]:
from main import *

forest = RFC(n_estimators=250)
forest.fit(train_images_hog, train_labels)

validate(forest, img_shift_x0y1_hog, "shifted by 1 px up image")
validate(forest, img_shift_x0y5_hog, "shifted by 5 px up image")
validate(forest, img_shift_x0y10_hog, "shifted by 10 px up image")
validate(forest, img_shift_x1y0_hog, "shifted by 1 px left image")
validate(forest, img_shift_x5y0_hog, "shifted by 5 px left image")
validate(forest, img_shift_x10y0_hog, "shifted by 10 px left image")
validate(forest, img_rotate_hog, "rotated image")
validate(forest, img_flip_x_hog, "flipped by Ox image")
validate(forest, img_flip_y_hog, "flipped by Oy image")

*** Validating shifted by 1 px up image took 133.2285032272339 s. Errors: 572 (5.72%)

*** Validating shifted by 5 px up image took 134.44955563545227 s. Errors: 8648 (86.48%)

*** Validating shifted by 10 px up image took 129.68616151809692 s. Errors: 8064 (80.64%)

*** Validating shifted by 1 px left image took 131.9364469051361 s. Errors: 385 (3.85%)

*** Validating shifted by 5 px left image took 134.39402151107788 s. Errors: 6817 (68.17%)

*** Validating shifted by 10 px left image took 135.1702218055725 s. Errors: 8715 (87.15%)

*** Validating rotated image took 116.79308915138245 s. Errors: 8668 (86.68%)

*** Validating flipped by Ox image took 120.44424533843994 s. Errors: 6562 (65.62%)

*** Validating flipped by Oy image took 118.01659893989563 s. Errors: 4998 (49.98%)



## Часть 3, добавление в обучающую выборку обработанных изображений

In [1]:
from main import *
        
extend(train_images_hog, train_labels, img_shift_x0y1_hog, test_labels)
extend(train_images_hog, train_labels, img_shift_x0y5_hog, test_labels)
extend(train_images_hog, train_labels, img_shift_x0y10_hog, test_labels)
extend(train_images_hog, train_labels, img_shift_x1y0_hog, test_labels)
extend(train_images_hog, train_labels, img_shift_x5y0_hog, test_labels)
extend(train_images_hog, train_labels, img_shift_x10y0_hog, test_labels)
extend(train_images_hog, train_labels, img_rotate_hog, test_labels)

forest = RFC(n_estimators=250)
forest.fit(train_images_hog, train_labels)

validate(forest, img_shift_x0y1_hog, "shifted by 1 px up image")
validate(forest, img_shift_x0y5_hog, "shifted by 5 px up image")
validate(forest, img_shift_x0y10_hog, "shifted by 10 px up image")
validate(forest, img_shift_x1y0_hog, "shifted by 1 px left image")
validate(forest, img_shift_x5y0_hog, "shifted by 5 px left image")
validate(forest, img_shift_x10y0_hog, "shifted by 10 px left image")
validate(forest, img_rotate_hog, "rotated image")

*** Validating shifted by 1 px up image took 115.17449927330017 s. Errors: 166 (1.66%)

*** Validating shifted by 5 px up image took 116.19420289993286 s. Errors: 673 (6.73%)

*** Validating shifted by 10 px up image took 113.92799019813538 s. Errors: 435 (4.35%)

*** Validating shifted by 1 px left image took 113.7607672214508 s. Errors: 204 (2.04%)

*** Validating shifted by 5 px left image took 113.65831351280212 s. Errors: 536 (5.36%)

*** Validating shifted by 10 px left image took 113.3451817035675 s. Errors: 369 (3.69%)

*** Validating rotated image took 113.50495958328247 s. Errors: 1012 (10.12%)



Как видно, добавление в обучающую выборку модифицированных изображений кардинально изменило результаты. Обучающая выборка была расширена на 35000 изображений: по 5000 из каждой категории (сдвиги, поворот). В результате удалось добиться не более 10% ошибок, в среднем же модель допускала 3-4% ошибок.